## Distributed Model Training

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: June 7, 2024

---

### TODO

1. Review the source papers and add relevant details

---

### SOURCES: 

- [Distributed Machine Learning Frameworks and its Benefits](https://www.xenonstack.com/blog/distributed-ml-framework#:~:text=In%20distributed%20machine%20learning%2C%20model,and%20training%20each%20split%20separately.)

- [Distributed Training with Azure](https://learn.microsoft.com/en-us/azure/machine-learning/concept-distributed-training?view=azureml-api-2/)

- [Distributed Training: Guide for Data Scientists
](https://neptune.ai/blog/distributed-training)

- Mastering Reinforcement Learning with Python, Enes Bilgin.

- [Distributed model training II: Parameter Server and AllReduce](http://www.juyang.co/distributed-model-training-ii-parameter-server-and-allreduce/)

Need to Review:

- [Distributed Machine Learning and the Parameter Server](https://www.cs.cornell.edu/courses/cs4787/2019sp/notes/lecture22.pdf)


### OBJECTIVES

- Explain approaches for distributed model training
- Identify the benefits and challenges of synchronous and asynchronous training
- Explain the parameter server and Allreduce algorithms
- Explain why Ring-Allreduce can work better than Allreduce 

### CONCEPTS

- Data parallelism and model parallelism
- Asynchronous training vs Synchronous training 
- Parameter server algorithm
- Allreduce algorithm and Ring-Allreduce algorithm
- Vertical partitioning and horizontal partitioning of a model

---

### I. Why use Distributed Model Training?


For massive training sets, it may not be possible to train a model on a single machine. 

This may be the case for deep learning models.

In *distributed training*, the workload to train a model is split up and shared among worker nodes. 

The concepts, benefits, and challenges are similar to what we've learned earlier.

The work can be parallelized to speed up training.

This process introduces benefits but also complexity.

---

### II. Data Parallelism and Model Parallelism

Two main types of distributed training: *data parallelism* and *model parallelism*.

#### Data Parallelism

This follows the approach used by Spark

Data is divided into partitions

Number of partitions = total number of available nodes

Model is copied in each worker node

Each node operates on its subset of data



<img src="./data_parallelism.png" width=600>

Each node must:

- Independently compute errors between training sample predictions and labels
- Update its model based on errors
- Communicate all of its changes to the other nodes to update their corresponding models

Worker nodes need to synchronize gradients at end of batch computation to ensure they're training a consistent model.

---

#### Model Parallelism

In some cases, the model may be too large for a worker

The strategy is to segment the model into different parts that run concurrently in different workers

Each model part runs on same data

Scalability depends on degree of task parallelization of algorithm

Worker nodes need to synchronize shared parameters, usually once for each forward or backward-propagation step 

More complex to implement than data parallelism

**Example of model parallelized on two GPUs**


<img src="./model_parallelism.png" width=400>

Model may be divided horizontally or vertically into different parts 

The parts run concurrently in different workers with each worker running on the same data

Worker needs to synchronize shared parameters, usually once for each forward or backward-propagation step

This **illustration** shows different partitions. Vertical partitioning keeps together all neurons in a layer, which works well for training deep learning models.  

<img src="./horiz_vert_part.png" width=400>

---

### III. Synchronous Training

Consider data parallelism case: 
- Data is divided into partitions
- Each partition is sent to a worker 
- Each worker has full replica of model. Training is done on its partition. 

**Forward Pass**

In synchronous training, forward pass begins at same time for each worker

Each worker computes different output and gradients

Each worker waits for the others to complete training loops and calculate respective gradients

After all workers have computed gradients, they communicate with each other and aggregate gradients using *Allreduce algorithm* (below)

After all gradients are combined, the updated gradients are copied to all workers. 

**Backward Pass**

Each worker performs backward pass and updates their local weights

Each worker will have different gradients as they are trained on different subsets of data

However, at any point in time, all workers have the same weights

---

#### Synchronous Algorithm Example: Allreduce Algorithm

Each node has a subset of the data 

Each node calculates the model's gradient and distributes it to the other nodes

Algorithm reduces the target arrays in all workers to a single array and returns the resultant array to all workers

Gradients are combined which updates model weights on each node

Notice from this example that the operation proceeds elementwise. The reduction is general (e.g., it might add the values).

<img src="allreduce.png" width=600>

source: https://tech.preferred.jp/en/blog/technologies-behind-distributed-deep-learning-allreduce/


There are different implementations:

**1. Naive Allreduce**

Each worker sends gradients to single worker called the *driver*.  

The driver reduces the gradients and sends updated gradients to all workers.  

**Challenge with this approach** is the driver is a bottleneck. As the number of nodes increases, communcation cost and reduction step scales poorly.

--

**2. Ring-Allreduce** 
  
Workers are set up in a ring (recall *consistent hashing*)  

Each worker is in charge of subset of parameters shared only with the next worker in the ring  

Less time spent sending data and more time spent doing computations on data locally on each GPU  

This allows for efficient averaging of gradients in neural networks across many devices and many nodes  

Retains the determinism and predictable convergence properties of synchronous stochastic gradient descent


<img src="ringreduce.png" width=600>

source: https://towardsdatascience.com/visual-intuition-on-ring-allreduce-for-distributed-deep-learning-d1f34b4911da

---

### IV. Asynchronous Training

We apply data parallelism here as well

Asynchronous training can be more efficient than synchronous training since there is no waiting. 

This is especially helpful when there is variation in the computing power across workers.

Thus in asynchronous training, we want workers to work independently in such a way that a worker need not wait for any other worker in the cluster. One way to achieve this is by using a parameter server.

#### Asynchronous Algorithm Example: Parameter Server

In this approach, weights and biases of ML model are distributed across nodes in cluster  

A copy of the model is stored on each node and a centralized *parameter server* manages model changes.  
This pattern is called *centralized training*.

Here is how it works:

1. Designate some nodes as parameter servers, while others train the model

- Parameter servers store model parameters, update global state of model
- Training workers run training loop, calculate gradients and loss. They each use subset of data.

2. Each training worker fetches parameters from parameter servers
3. Each training worker performs training loop, sends gradients back to all parameter servers, which updates model parameters





<img src="./parameter_server.png" width=300>

**Challenges with this approach**

At any given time, only one worker is using updated version of model. Others are using stale version. 

If one worker is used as parameter server, it can be a bottleneck and single point of failure.  
Using multiple parameter servers alleviates this issue but adds communication costs.


**Parameter Server Example: Gorila - General Reinforcement Learning Architecture**

*Source: Mastering Reinforcement Learning with Python, Enes Bilgin.*

Background:  
- Reinforcement learning (RL) requires massive training datasets. There is no label used; instead, a reward function is specified.
- An agent exists in some state (e.g., an investor holds a portfolio of stocks) and must take a series of actions over time (buy/sell orders).
- Everything outside of the agent is the environment (the economy, the stock markets)
- We might have historical data (experiences) that can be used
- The goal is to learn the best action to take given each state. This is the *optimal policy*. It needs to maximize long-term reward.
- In Deep RL, a common approach is to use a neural network for learning the optimal policy. The *Q-network* is one approach.


Strategy: we need to simulate experiences, and we can parallelize this. We store experiences in a replay buffer.

Here are the important components:

**Actors:**  
Processes that interact with copy of environment given a policy, take the action, observe the reward, and transition to the next state.

Actors get a copy of the Q-network from a parameter server

**Replay Buffer:**  
Actors collect experiences and store them in a replay buffer. This can be done in a distributed fashion.

**Learners:**  
Learners calculate the gradients that update the Q-network in the parameter server.  
A learner has a copy of the Q-network, it samples experiences from the buffer, calculates the loss and gradients, and sends them back to the parameter server

**Parameter Server:**  
The parameter server stores the main copy of the Q-network. As learning progresses, updates happen here.    
All processes sync their version of the Q-network from here.

--

**Next, we provide an overview of Gorila**

Gorila provides a general framework to parallelize deep Q-learning

It consists of bundles which can be distributed to workers, where each bundle comprises an action, learner, and local replay buffer.  

Next we show an illustration of the approach. For our purposes, the details are unimportant; the takeaway is how the parameter server supports learning.

<img src="./gorila.png" width=600>

Notice how the learners send gradients back to the parameter server

The learners get the synced Q-network from the parameter server

**Shortcoming:** There is a lot of passing around of parameters between the actors, learners, and parameter server.  
This can be a significant communication load.

If you are interested, you can [explore](https://arxiv.org/pdf/1803.00933) how the *Ape-X* architecture improves Gorila!



---

### V. Implementation

**Elephas** is a Keras add-on that allows you to use Spark to execute distributed deep learning models at scale.

**Amazon SageMaker** offers 

**Horovod**. Open-source distributed training framework developed at Uber for TensorFlow, Keras, PyTorch, and MXNet.  
Supports practical distributed training over several GPUs and nodes.


---

### VI. Conclusions

We have studied different approaches for distributed model training

Benefits can include scalability, efficiency, and fault tolerance 

These come at the cost of increased network communication load

Data parallelism (partitioning the data across nodes) is easier than model parallelism

Model parallelism is useful when the model won't fit on a single machine. 

Vertical partitioning is easier for deep learning, as layers are kept together on workers

There are several packages that implement distributed model training

**Tradeoffs**

As always, there are tradeoffs to the methods:

- Centralized training, such as the parameter server approach, can introduce a communication bottleneck. More parameter servers can be added to reduce the bottleneck.

- The parameter server approach is asynchronous which can increase efficiency: there is no waiting for workers. This is especially helpful when the nodes have differences in processing power.


---